CSE 5243 

# Lab 2: k-Nearest Neighbors

Feb. 12th, 2018

## 1. Implement a Custom Proximity Function

In [6]:
import pandas as pd
import numpy as np
df=pd.read_csv("lab2/income_tr.csv", sep=',', na_values=[" ?"],engine='python')

## Proximity Function

The Proximity Function is calculating the mean square of the distances of different attributes between two subjects as **dissimilarity**. For a continuous attribute, the distance is defined as *(Vsubj1 - Vsubj2) / RofAttr*. Howerver, for a nominal attribute, the distance is 0 if the value matches or 1 if the values do not match. The function is implemented as following.

In [7]:
r={}
# Calculate and Store the Range for every columns
for item in df.columns:
    if type(df[item][0])==type((df.loc[0])['age']):
        r[item]=df[item].max()-df[item].min()
        

def dissimilarity(ob1,ob2):
    '''
    function for calculating dissimilarity
    '''
    dis=0
    sum=0
    length=0
    for item in df.columns:
        if item!='ID' and item!='class':
            length+=1
            if type(ob1[item])==type((df.loc[0])['age']):
                dis=(ob1[item]-ob2[item])/r[item]
            elif ob1[item]==ob2[item]:
                dis=0
            else:
                dis=1
        sum+=dis*dis
    dissimilarity=sum/length
    return dissimilarity

The function cul is for calculating the distance for every 2 subjects in the training dataset. 

In [8]:
def cul(k):
    labels=[]
    simMat=np.zeros((len(df),len(df)))
    for i in range(0,2*k):
        ii=np.floor(i/2)
        if i%2==0:
            labels.append(str(int(ii+1))+'st')
        else:
            labels.append(str(int(ii+1))+'-prox')
    output=pd.DataFrame(columns=labels)
    for i in range(0,len(df)):
        if False and i%15==0:  # switch to True if the running progress is needed
            print(str(i) + "/" + str(len(df)) + "done")
        sim=[[],[]]
        length=0
        for j in range(0,len(df)):
            # culculate and sort dissimilarity
            if i==j:
                dis=0;
            elif i>j:
                dis=simMat[j][i]
            else:
                dis=dissimilarity(df.loc[i],df.loc[j]) 
                simMat[i][j]=dis
            kk=0
            for l in range(0,length):
                if dis>sim[0][l]:
                    kk=l+1
                else:
                    break
            length+=1
            sim[0].insert(kk,dis)
            sim[1].insert(kk,j)
            
        #choose the first ks    
        newRow=[]
        for i in range(0,k):
            newRow.append(sim[1][i+1])
            newRow.append(sim[0][i+1])
                
        newdf=pd.DataFrame(data=[newRow],columns=labels)
        output=output.append(newdf,ignore_index=True)
        
    return output
simdf=cul(4)

KeyboardInterrupt: 

This is a sample for the output with k=4

In [ ]:
simdf.sample(5)

In [ ]:
simdf.to_csv('output_part1.csv',sep=',')

## 2. kNN Classifier

In this section, we implement a k-nearest neighbors classifier based on the proximity function described in section 1. The prediction performance is also evaluated with various metrics. 

For the outputs shown in this section, we set the system parameters to be k=5, proximity threshold = 0.5, positive class = ' >50K', if not otherwise specified.

### 2.1. Implement a kNN Classifier

First, we import the test set. A sample of 5 records of the test set is as follows.

In [ ]:
te = pd.read_csv("income_te.csv", sep=',', na_values=[" ?"],engine='python')
te.sample(5)

The kNN classifier is implemented with the following two functions. The function $knnDists$ takes the training set $tr$, test set $te$, and k value as input, and returns a dictionary storing the distances between data in the training set and data in the test set. The function $knnClassify$ classifies all the records in the test set based on their distances with k nearest neighbors in the training set. It takes the training and test dataset ($te$ and $tr$), the distance dictionary from $knnDists$, the interested attribute $targetClass$, the proximity threshold $t$ and its corresponding class value $tClass$ as input, and outputs the classification results.

In [ ]:
def knnDists(te, tr, k=5):   
    res = {}
    for iTe in range(0,len(te)):
        if True and iTe%15==0:  # switch to True if the running progress is needed
            print(str(iTe) + "/" + str(len(te)) + "done")
        distDic = {}
        distMax = None # current max dist
        for iTr in range(0,len(tr)):
            dist = dissimilarity(te.loc[iTe], tr.loc[iTr]) 
            # not a great improvement
            if len(distDic)<k: 
                distDic[iTr] = dist
                if distMax==None or dist>distMax:
                    distMax = dist
                continue
            if dist<distMax:
                distDic[iTr] = dist
            # distDic[iTr] = dist
        knnDistDic = {}
        for iKey in sorted(distDic, key=distDic.get):
            knnDistDic[iKey] = distDic[iKey]
            if len(knnDistDic)>=k:
                break
        res[iTe] = knnDistDic        
    return res


def knnClassify(te, tr, knnDistDic, targetClass, k=5, tClass=' >50K', t=0.5):
    res = {}
    for iTe in range(0,len(te)):
        iTeRes = {}
        classDic = {}
        knn = knnDistDic[iTe]
        if len(knn)>k:
            knn = sorted(knn,key=knn.get)[0:k]
        for iTr in knn:
            if tr.loc[iTr][targetClass] not in classDic:
                classDic[tr.loc[iTr][targetClass]] = 1
            else:
                classDic[tr.loc[iTr][targetClass]] += 1
        iTeRes['class'] = sorted(classDic, key=classDic.get, reverse=True)[0]
        iTeRes['prob'] = classDic[iTeRes['class']]/sum(classDic.values()) 
        if iTeRes['prob']<t:
            iTeRes['class'] = sorted(classDic, key=classDic.get, reverse=True)[1]
            iTeRes['prob'] = classDic[iTeRes['class']]/sum(classDic.values()) 
        res[iTe] = iTeRes
    return res        

In [ ]:
# be patient
knnDistDic = knnDists(te, df)

In [19]:
knnClasses = knnClassify(te, df, knnDistDic, 'class')

Get the required format and export the results as a csv file.

In [20]:
def formatOutput(te, knnResDic, targetClass):
    labels = ['Transaction ID', 'Actual Class', 'Predicted Class', 'Posterior probability']
    output = pd.DataFrame(columns=labels)
    for iTe in range(0,len(te)):
        row = [iTe,]
        row.append(te.loc[iTe][targetClass])
        row.append(knnResDic[iTe]['class'])
        row.append(knnResDic[iTe]['prob'])
        output = output.append(pd.DataFrame(data=[row], columns=labels), ignore_index=True)
    return output

In [21]:
knnOutput = formatOutput(te, knnClasses, 'class')
knnOutput.to_csv('output_part2.csv',sep=',',index=False)

A sample of 5 records of the output is as follows:

In [22]:
knnOutput.sample(5)

,Transaction ID,Actual Class,Predicted Class,Posterior probability
188,188,<=50K,<=50K,1.0
51,51,<=50K,<=50K,1.0
64,64,<=50K,<=50K,0.8
242,242,<=50K,<=50K,0.8
86,86,>50K,>50K,1.0


### 2.2. Performance evaluation
In this section, we will evaluation the system performance with various metrics.

First, we count the true positives, true negatives, false positives, and false negatives; and then calculate the corresponding rates.

In [23]:
# Counts
def countStat(knnOutput, posClass):
    stat = {} # for true positive, true negative, false positive, and false negative
    for iTe in range(0,len(knnOutput)):
        if knnOutput.iloc[iTe]["Predicted Class"] == posClass:
            if knnOutput.iloc[iTe]["Actual Class"] == posClass:
                if 'true positive' not in stat:
                    stat['true positive'] = 1
                else:
                    stat['true positive'] += 1
            else:
                if 'false positive' not in stat:
                    stat['false positive'] = 1
                else:
                    stat['false positive'] += 1
        else:
            if knnOutput.iloc[iTe]["Actual Class"] == posClass:
                if 'false negative' not in stat:
                    stat['false negative'] = 1
                else:
                    stat['false negative'] += 1
            else:
                if 'true negative' not in stat:
                    stat['true negative'] = 1
                else:
                    stat['true negative'] += 1
    return stat

                    
stat = countStat(knnOutput, ' >50K')

# Calculate the rates
for i in stat:
    print(i + " count = " + str(stat[i]))
    print(i + " rate = " + str(round(stat[i]/sum(stat.values())*100,2)) + "%")

true negative count = 190
true negative rate = 65.97%
false negative count = 43
false negative rate = 14.93%
true positive count = 24
true positive rate = 8.33%
false positive count = 31
false positive rate = 10.76%


Thus the confusion matrix is,

|              |Predicted Class: >50K|Predicted Class: <=50K|
|-----------------------------------------------------------------|
|Actual Class: >50K  |24             |43              |
|Actual Class: <=50K |31             |190             |

And the accuracy and error rate are,

In [24]:
accuracy = (stat['true positive']+stat['true negative']) / sum(stat.values())
errorRate = (stat['false positive']+stat['false negative']) / sum(stat.values())

print("accuracy = " + str(round(accuracy*100,2)) + "%")
print("error rate = " + str(round(errorRate*100,2)) + "%")

accuracy = 74.31%
error rate = 25.69%


The prediction precision, recall, and $F_1$ score can be calculated as following:

In [25]:
precision = stat['true positive'] / (stat['true positive']+stat['false positive'])
recall = stat['true positive'] / (stat['true positive']+stat['false negative'])
f1 = 2 * (precision*recall) / (precision+recall)

print("precision = " + str(round(precision*100,2)) + "%")
print("recall = " + str(round(recall*100,2)) + "%")
print("F1 score = " + str(round(f1*100,2)) + "%")

precision = 43.64%
recall = 35.82%
F1 score = 39.34%


From the above metrics, we can see that the prediction result is not ideal. One possible cause is the positive-class records (annual income > 50K) only take a small portion of both the training set and the test set. More negative data may increase the noise for prediction.

## 3. Exceptional Work

### Optimal Value K

In this section, we study how the prediction results will change with different values for k.

In [26]:
knnDistDic = knnDists(te, df, 10)
for k in range(1, 11):
    print("k = " + str(k) + ":")    
    knnClasses = knnClassify(te, df, knnDistDic, 'class',k)
    knnOutput = formatOutput(te, knnClasses, 'class')
    stat = countStat(knnOutput, ' >50K')
    print(stat)

k = 1:
{'true negative': 182, 'false negative': 40, 'false positive': 39, 'true positive': 27}
k = 2:
{'true negative': 182, 'false negative': 40, 'false positive': 39, 'true positive': 27}
k = 3:
{'false positive': 35, 'true negative': 186, 'false negative': 36, 'true positive': 31}
k = 4:
{'true negative': 190, 'false negative': 41, 'false positive': 31, 'true positive': 26}
k = 5:
{'true negative': 190, 'false negative': 43, 'true positive': 24, 'false positive': 31}
k = 6:
{'true negative': 190, 'false negative': 43, 'true positive': 24, 'false positive': 31}
k = 7:
{'false positive': 26, 'true negative': 195, 'false negative': 45, 'true positive': 22}
k = 8:
{'true negative': 195, 'false negative': 44, 'true positive': 23, 'false positive': 26}
k = 9:
{'true negative': 197, 'false negative': 42, 'true positive': 25, 'false positive': 24}
k = 10:
{'true negative': 196, 'false negative': 44, 'true positive': 23, 'false positive': 25}


The evaluation metrics for different k's are shown in the following table:

| | True positive rate | True negative rate | False positive rate | False negative rate | Accuracy | Error rate | Precision | Recall | F1 score |
|-----|
|k=1| 9.38% | 63.19% | 13.54% | 13.89% | 72.57% | 27.43% | 40.91% | 40.3% | 40.6% |
|k=2| 9.38% | 63.19% | 13.54% | 13.89% | 72.57% | 27.43% | 40.91% | 40.3% | 40.6% |
|k=3| 10.76% | 64.58% | 12.15% | 12.5% | 75.35% | 24.65% | 46.97% | 46.27% | 46.62% |
|k=4| 9.03% | 65.97% | 10.76% | 14.24% | 75.0% | 25.0% | 45.61% | 38.81% | 41.94% |
|k=5| 8.33% | 65.97% | 10.76% | 14.93% | 74.31% | 25.69% | 43.64% | 35.82% | 39.34% |
|k=6| 8.33% | 65.97% | 10.76% | 14.93% | 74.31% | 25.69% | 43.64% | 35.82% | 39.34% |
|k=7| 7.64% | 67.71% | 9.03% | 15.62% | 75.35% | 24.65% | 45.83% | 32.84% | 38.26% | 
|k=8| 7.99% | 67.71% | 9.03% | 15.28% |75.69% | 24.31% | 46.94% | 34.33% | 39.66% | 
|k=9| 8.68% | 68.4% | 8.33% | 14.58% | 77.08% | 22.92% |  51.02%  | 37.31% | 43.1% | 
|k=10|7.99% | 68.06% | 8.68% | 15.28% | 76.04% | 23.96% | 47.92% | 34.33% | 40.0% | 

From the above table, it is obvious that the choice of k will effect the prediction performance. For k's from 1 to 10, k=3 produces the highest true positive rate and the highest F1 socre 46.62%.

### Generalize Proximity Function

Change the function *dissimilarity* to *Gdis* that can be applied to any data set. The input variables are:
* i1: The index for the first comparing subject.
* i2: The index for the second comparing subject.
* df: The dataframe in which the subjects locate.
* exList: Name of the Columns that are exluded when calcualting dissimilarity.
* dataSetName: Choose a name for the dataset. (Note that different dataframe should have different name.)

In [86]:
r={}
def Gdis(i1,i2, df, exList=[],dataSetName):
    # Calculate and Store the Range for every columns
    if dataSetName not in r:
        r[dataSetName]={}
        for item in df.columns:
            if isinstance(df[item][0],np.int64):
                r[dataSetName][item]=df[item].max()-df[item].min()
    '''
    function for calculating dissimilarity
    '''
    ob1=df.loc[i1]
    ob2=df.loc[i2]
    dis=0
    sum=0
    length=0
    for item in df.columns:
        if item not in exList:
            length+=1
            if isinstance(ob1[item],np.number):
                dis=(ob1[item]-ob2[item])/r[dataSetName][item]
            elif ob1[item]==ob2[item]:
                dis=0
            else:
                dis=1
        sum+=dis*dis
    dissimilarity=sum/length
    return dissimilarity

This function works for the original income dataset.

In [87]:
Gdis(0,30,df,['ID','class'],'income')

0.0047252527132368187

And also for a new dataset with different attributes.

In [107]:
df2 = pd.DataFrame({ 'A' : 1.,
   ....:                      'B' : pd.Timestamp('20130102'),
   ....:                      'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
   ....:                      'D' : np.array([3] * 4,dtype='int32'),
   ....:                      'E' : pd.Categorical(["test","train","test","train"]),
   ....:                      'F' : 'foo', 
   ....:                      'G' : [1,2,3,4]})
df2

,A,B,C,D,E,F,G
0,1.0,2013-01-02,1.0,3,test,foo,1
1,1.0,2013-01-02,1.0,3,train,foo,2
2,1.0,2013-01-02,1.0,3,test,foo,3
3,1.0,2013-01-02,1.0,3,train,foo,4


In [108]:
Gdis(0,1,df2,[],'example')

0.15873015873015875